## Inputs

In [13]:
from numpy.lib.shape_base import split
import trader_tool as tt
import pandas as pd
import numpy as np
from fpdf import FPDF
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import tqdm as tq
import glob
import datetime

In [14]:
all_prices = glob.glob(r'S:\2022Candles\82122spreadonly\*.csv')
# data from trading view


frequency_min = 15
frequency_max = 25

upper_min = 70
upper_max = 80

lower_min = 20
lower_max = 30


initial_capital = 100000
commission = 0
start = 0

type_of_commission = "Reduce Contracts" 
Close_position = True

# Query Inputs
query = 'Profit factor'
sorting_params = ["Net Profit", "% Profitability", "Average Win/Loss", "Profit factor", "Profit Power"]
boundary = 1.3
reduce_1 = 200
reduce_2 = 500
min_trades = 20
max_trades = 100


print(*all_prices,sep='\n')

S:\2022Candles\82122spreadonly\CBOT_MINI MYM1!_CME_MINI MNQ1!, 1.csv
S:\2022Candles\82122spreadonly\CBOT_MINI MYM1!_CME_MINI MNQ1!, 2.csv
S:\2022Candles\82122spreadonly\CBOT_MINI MYM1!_CME_MINI MNQ1!, 3.csv
S:\2022Candles\82122spreadonly\CBOT_MINI MYM1!_CME_MINI MNQ1!, 5.csv


## Calculate RSX

In [15]:
ranges = [frequency_max-frequency_min+1, upper_max-upper_min+1, lower_max-lower_min+1]

col_names = ["Frequency","Upper","Lower","Net Profit",	"Total Trades",	"% Profitability",	"Average Win/Loss",	
             "Average Trade", "Average Bars",	"Profit factor",	"Stratergy Max Drawdown", "Profit Power",	 
             "Min Runup",	"Average Runup",	"Max Runup", "Min Drawdown", "Average Drawdown",	"Max Drawdown", 
             "Cumalative Profit"]
df_both    = pd.DataFrame(0,index= np.arange(ranges[0] * ranges[1] * ranges[2]),columns=col_names)
df_long    = pd.DataFrame(0,index= np.arange(ranges[0] * ranges[1] * ranges[2]),columns=col_names)
df_short   = pd.DataFrame(0,index= np.arange(ranges[0] * ranges[1] * ranges[2]),columns=col_names)


def rsx_full(file):
    prices = pd.read_csv(file, usecols=["open","high","low","close"])    
    r = 0
    for frequency in tq.tqdm(range(frequency_min, frequency_max + 1)):    

        rsx_full = tt.rsx(frequency, prices["close"])
        if start > 2:
            rsx_full = rsx_full[start-2:]
            prices = prices[start-2:]
        
        for Upper in range(upper_min, upper_max+1):
            for Lower in range(lower_min, lower_max+1):
                metr_both, metr_long, metr_short = tt.trade_triggers(Upper, Lower, rsx_full, initial_capital, 
                                                                    prices, commission, Close_position=Close_position, 
                                                                    type_of_commission=type_of_commission)
                
                df_both.iloc[r,:]  = np.append([frequency, Upper, Lower], metr_both)
                df_long.iloc[r,:]  = np.append([frequency, Upper, Lower], metr_long)
                df_short.iloc[r,:] = np.append([frequency, Upper, Lower], metr_short)
                r += 1
    
    return df_both, df_long, df_short
    
    

## Queries

In [16]:
# Collect data 
def collect_details(df_details,  df_options, information):
    
    return 
    

In [17]:
def all_queries(df_both, df_long, df_short):
    # Remove too many and too frew trades
    df_both_r    = df_both.loc[(df_both['Total Trades'] >= min_trades) & (df_both['Total Trades'] <= max_trades)]
    df_long_r    = df_long.loc[(df_long['Total Trades'] >= min_trades) & (df_long['Total Trades'] <= max_trades)]
    df_short_r   = df_short.loc[(df_short['Total Trades'] >= min_trades) & (df_short['Total Trades'] <= max_trades)]

    # Reduce 1
    df_both_options_r1 = tt.simple_query(df_both_r, query, boundary, sorting_params, reduce_1)
    df_long_options_r1 = tt.simple_query(df_long_r, query, boundary, sorting_params, reduce_1)
    df_short_options_r1 = tt.simple_query(df_short_r, query, boundary, sorting_params, reduce_1)
    
    return df_both_options_r1, df_long_options_r1, df_short_options_r1

In [18]:
both_options_r1 = {}
long_options_r1 = {}
short_options_r1 = {}

for i in range(len(all_prices)):
    print(all_prices[i])
    df_both, df_long, df_short = rsx_full(all_prices[i])
    df_both_options_r1, df_long_options_r1, df_short_options_r1 = all_queries(df_both=df_both, df_long=df_long, df_short=df_short)
    name_parts = all_prices[i].split(', ')
    timeframe = name_parts[1].split('.')[0]
    name = name_parts[0].split(' ')[1] + ' ' + timeframe

    both_options_r1[name] = df_both_options_r1.loc[:, ('Sort by', 'Frequency', 'Upper', 'Lower')]
    long_options_r1[name] = df_long_options_r1.loc[:, ('Sort by', 'Frequency', 'Upper', 'Lower')]
    short_options_r1[name] = df_short_options_r1.loc[:, ('Sort by', 'Frequency', 'Upper', 'Lower')]

big_both = pd.concat(both_options_r1, axis=1)
big_long = pd.concat(long_options_r1, axis=1)
big_short = pd.concat(short_options_r1, axis=1)

S:\2022Candles\82122spreadonly\CBOT_MINI MYM1!_CME_MINI MNQ1!, 1.csv


100%|██████████| 11/11 [00:32<00:00,  2.92s/it]


S:\2022Candles\82122spreadonly\CBOT_MINI MYM1!_CME_MINI MNQ1!, 2.csv


100%|██████████| 11/11 [00:33<00:00,  3.08s/it]


S:\2022Candles\82122spreadonly\CBOT_MINI MYM1!_CME_MINI MNQ1!, 3.csv


100%|██████████| 11/11 [00:34<00:00,  3.17s/it]


S:\2022Candles\82122spreadonly\CBOT_MINI MYM1!_CME_MINI MNQ1!, 5.csv


100%|██████████| 11/11 [00:36<00:00,  3.34s/it]


In [19]:
print('Both')
print(big_both.to_markdown())
print('\n\n')
print('Long')
print(big_long.to_markdown())
print('\n\n')
print('Short')
print(big_short.to_markdown())

Both
|    | ('MYM1!_CME_MINI 1', 'Sort by')   |   ('MYM1!_CME_MINI 1', 'Frequency') |   ('MYM1!_CME_MINI 1', 'Upper') |   ('MYM1!_CME_MINI 1', 'Lower') | ('MYM1!_CME_MINI 2', 'Sort by')   |   ('MYM1!_CME_MINI 2', 'Frequency') |   ('MYM1!_CME_MINI 2', 'Upper') |   ('MYM1!_CME_MINI 2', 'Lower') | ('MYM1!_CME_MINI 3', 'Sort by')   |   ('MYM1!_CME_MINI 3', 'Frequency') |   ('MYM1!_CME_MINI 3', 'Upper') |   ('MYM1!_CME_MINI 3', 'Lower') | ('MYM1!_CME_MINI 5', 'Sort by')   |   ('MYM1!_CME_MINI 5', 'Frequency') |   ('MYM1!_CME_MINI 5', 'Upper') |   ('MYM1!_CME_MINI 5', 'Lower') |
|---:|:----------------------------------|------------------------------------:|--------------------------------:|--------------------------------:|:----------------------------------|------------------------------------:|--------------------------------:|--------------------------------:|:----------------------------------|------------------------------------:|--------------------------------:|----------------------

## Pine Scipt Output

In [20]:
# Delete 
file_read = open("TV_input.txt", "r")
lines = file_read.readlines()

# New infor
def pinescript(title, big_long, big_both, file_append):
    all_info = title.split(', ')
    trade = all_info[0]
    trade = trade.split(" ")[1]
    time_frame = all_info[-1].split('.')[0]
    name = all_info[0].split(' ')[1] + ' ' + time_frame

    
    # Format
    template = """
//Trade : {name}
if syminfo.ticker == "{trade}" and timeframe.period == "{time_frame}"
    is_scanned := 1
    primary_per_length := {primary_per_length}
    primary_per_obLevel := {primary_per_obLevel}
    primary_per_osLevel := {primary_per_osLevel}
    primary_net_length := {primary_net_length}
    primary_net_obLevel := {primary_net_obLevel}
    primary_net_osLevel := {primary_net_osLevel}
    primary_awl_length := {primary_awl_length}
    primary_awl_obLevel := {primary_awl_obLevel}
    primary_awl_osLevel := {primary_awl_osLevel}
    primary_pf_length := {primary_pf_length}
    primary_pf_obLevel := {primary_pf_obLevel}
    primary_pf_osLevel := {primary_pf_osLevel}
    primary_ef_length := {primary_ef_length}
    primary_ef_obLevel := {primary_ef_obLevel}
    primary_ef_osLevel := {primary_ef_osLevel}
    secondary_per_length := {secondary_per_length}
    secondary_per_obLevel := {secondary_per_obLevel}
    secondary_per_osLevel := {secondary_per_osLevel}
    secondary_net_length := {secondary_net_length}
    secondary_net_obLevel := {secondary_net_obLevel}
    secondary_net_osLevel := {secondary_net_osLevel}
    secondary_awl_length := {secondary_awl_length}
    secondary_awl_obLevel := {secondary_awl_obLevel}
    secondary_awl_osLevel := {secondary_awl_osLevel}
    secondary_pf_length := {secondary_pf_length}
    secondary_pf_obLevel := {secondary_pf_obLevel}
    secondary_pf_osLevel := {secondary_pf_osLevel}
    secondary_ef_length := {secondary_ef_length}
    secondary_ef_obLevel := {secondary_ef_obLevel}
    secondary_ef_osLevel := {secondary_ef_osLevel} \n
    
//End of {name}
"""
    context = {
        "name" : name,
        "trade" : trade,
        "time_frame": time_frame,
        "primary_per_length" : int(big_both.loc[1, (name, 'Frequency')]),
        "primary_per_obLevel" : int(big_both.loc[1, (name, 'Upper')]),
        "primary_per_osLevel" : int(big_both.loc[1, (name, 'Lower')]),
        "primary_net_length" : int(big_both.loc[0, (name, 'Frequency')]),
        "primary_net_obLevel" : int(big_both.loc[0, (name, 'Upper')]),
        "primary_net_osLevel" : int(big_both.loc[0, (name, 'Lower')]),
        "primary_awl_length" : int(big_both.loc[2, (name, 'Frequency')]),
        "primary_awl_obLevel" : int(big_both.loc[2, (name, 'Upper')]),
        "primary_awl_osLevel" : int(big_both.loc[2, (name, 'Lower')]),
        "primary_pf_length" : int(big_both.loc[3, (name, 'Frequency')]),
        "primary_pf_obLevel" : int(big_both.loc[3, (name, 'Upper')]),
        "primary_pf_osLevel" : int(big_both.loc[3, (name, 'Lower')]),
        "primary_ef_length" : int(big_both.loc[4, (name, 'Frequency')]),
        "primary_ef_obLevel" : int(big_both.loc[4, (name, 'Upper')]),
        "primary_ef_osLevel" : int(big_both.loc[4, (name, 'Lower')]),
        "secondary_per_length" : int(big_long.loc[1, (name, 'Frequency')]),
        "secondary_per_obLevel" : int(big_long.loc[1, (name, 'Upper')]),
        "secondary_per_osLevel" : int(big_long.loc[1, (name, 'Lower')]),
        "secondary_net_length" : int(big_long.loc[0, (name, 'Frequency')]),
        "secondary_net_obLevel" : int(big_long.loc[0, (name, 'Upper')]),
        "secondary_net_osLevel" : int(big_long.loc[0, (name, 'Lower')]),
        "secondary_awl_length" : int(big_long.loc[2, (name, 'Frequency')]),
        "secondary_awl_obLevel" : int(big_long.loc[2, (name, 'Upper')]),
        "secondary_awl_osLevel" : int(big_long.loc[2, (name, 'Lower')]),
        "secondary_pf_length" : int(big_long.loc[3, (name, 'Frequency')]),
        "secondary_pf_obLevel" : int(big_long.loc[3, (name, 'Upper')]),
        "secondary_pf_osLevel" : int(big_long.loc[3, (name, 'Lower')]),
        "secondary_ef_length" : int(big_long.loc[4, (name, 'Frequency')]),
        "secondary_ef_obLevel" : int(big_long.loc[4, (name, 'Upper')]),
        "secondary_ef_osLevel" : int(big_long.loc[4, (name, 'Lower')])
        }

    file_append.write(template.format(**context))

# Repeat Info
def repeat_text(start, end, lines):
    file_write = open("TV_input.txt","w")
    for n, line in enumerate(lines):
        if start <= n and end >= n:
            file_write.write(line)

    file_write.close()
    

# Delete 
def remove_trade(first_ind, last_ind, lines):
    file_write = open("TV_input.txt","w")
    for n, line in enumerate(lines):
        if first_ind > n >= last_ind:
            file_write.write(line)            

    file_write.close()
    


titles = [title.split('\\')[-1] for title in all_prices ]
all_info = titles[0].split(', ')
trade = all_info[0]
first_ind = lines.index('//////////BEGIN ENTERING INPUTS\n')
last_ind = lines.index('//////////END ENTERING INPUTS\n')

# Remove Duplication
remove_trade(first_ind, last_ind, lines)

# # Start
repeat_text(0, first_ind, lines)
file_append = open("TV_input.txt", "a")
file_append.write(('//Time stamp: ' + str(datetime.datetime.now())))

# Pine Script
for title in titles:
    pinescript(title, big_long, big_both, file_append)
    
# End Remark
for line in lines[last_ind:]:
    file_append.write(line)
file_append.close()

## Visualized

In [21]:
# def vis_results(df_options, df):
#     rows = int(np.ceil(len(df_options)/2))
#     cols = 2
#     fig, axs = plt.subplots(nrows= rows, ncols=cols, figsize=(rows*7,cols*8))
#     ax = axs.ravel()
#     for i in range(len(df_options)):
#         cm_profit = df['Cumalative Profit'].loc[(df["Frequency"] == int(df_options["Frequency"].iloc[i]))
#                                                  & (df["Upper"] == int(df_options["Upper"].iloc[i]))
#                                                  & (df["Lower"] == int(df_options["Lower"].iloc[i]))]
        
#         cm_profit = cm_profit.values[0][0]+initial_capital
#         ax[i].plot(cm_profit)
#         ax[i].plot([0, len(cm_profit)],[initial_capital, initial_capital])
#         ax[i].set_xlabel('Trade')
#         ax[i].set_ylabel('Capital')
#         ax[i].set_title(('Equity Curve (' + df_both_options_r1["Frequency"].iloc[i] 
#                         + ', ' + df_both_options_r1["Upper"].iloc[i] + ', ' 
#                         + df_both_options_r1["Lower"].iloc[i] + ') : ' 
#                         + df_both_options_r1["Sort by"].iloc[i] ))

### Both

In [22]:
# vis_results(df_both_options_r1, df_both)
# print('Top 200 Results')
# print(df_both_options_r1.to_markdown())

In [23]:
# vis_results(df_both_options_r2, df_both)
# print('Top 500 Results')
# print(df_both_options_r2.to_markdown())

### Long

In [24]:
# vis_results(df_long_options_r1, df_long)
# print('Top 200 Results')
# print(df_long_options_r1.to_markdown())

In [25]:
# vis_results(df_long_options_r2, df_long)
# print('Top 500 Results')
# print(df_long_options_r2.to_markdown())

### Short

In [26]:
# vis_results(df_short_options_r1, df_short)
# print('Top 200 Results')
# print(df_short_options_r1.to_markdown())

In [27]:
# vis_results(df_short_options_r2, df_long)
# print('Top 500 Results')
# print(df_short_options_r2.to_markdown())